In [1]:
import sys
import dlib
import cv2
import os

from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K

import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf

from keras.models import load_model
from keras.utils import CustomObjectScope

%matplotlib inline
%load_ext autoreload
%autoreload 2

#Local Files
import align_dlib #Copied from https://github.com/cmusatyalab/openface/blob/master/openface/align_dlib.py

Using TensorFlow backend.


In [2]:
# Create a HOG face detector using the built-in dlib class
face_detector = dlib.get_frontal_face_detector()

#Facial Landmark Detector
# You can download the required pre-trained model from  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
predictor_model = "models/shape_predictor_68_face_landmarks.dat"
face_pose_predictor = dlib.shape_predictor(predictor_model)

#Align the faces using align_dlib.py
face_aligner = align_dlib.AlignDlib(predictor_model)

In [3]:
with CustomObjectScope({'tf': tf}):
    FRmodel = load_model('./models/nn4.small2.lrn.h5')

Instructions for updating:
Colocations handled automatically by placer.


/Users/shehan/python3_ml/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
def img_to_encoding_2(img1, model):
    #img = img1[...,::-1]
    img = np.around(np.transpose(img1, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

In [5]:
#Load database from database.npy
database=np.load("database.npy").item()

In [6]:
def who_is_it_2(img1, database, FRmodel):
    """    
    Arguments:
    img1 -- image
    database -- database containing image encodings along with the name of the person on the image
    FRmodel -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between img1 encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    
    ### START CODE HERE ### 
    
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding_2() see example above. ## (≈ 1 line)
    encoding = img_to_encoding_2(img1, FRmodel)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current "emb" from the database. (≈ 1 line)
        dist = np.linalg.norm((db_enc - encoding))

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if dist<min_dist:
            min_dist = dist
            identity = name

    ### END CODE HERE ###
    
    if min_dist > 0.8:
        print("Not in the database, the distance is " + str(min_dist) )
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist) )
        
    return min_dist, identity

In [7]:
def recognize_faces_in_image(file_name):
    image= cv2.imread(file_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    win = dlib.image_window()
    win.set_image(image)
    
    detected_faces = face_detector(image, 1)
    
    for i, face_rect in enumerate(detected_faces):

        # Draw a box around each face we found
        win.add_overlay(face_rect)

        # Get the the face's pose
        pose_landmarks = face_pose_predictor(image, face_rect)

        # Use openface to calculate and perform the face alignment
        alignedFace = face_aligner.align(96, image, face_rect, landmarkIndices=align_dlib.AlignDlib.OUTER_EYES_AND_NOSE)

        who_is_it_2(alignedFace, database, FRmodel)
    
    #dlib.hit_enter_to_continue()
        
        

In [8]:
#recognize_faces_in_image('images_original/1.jpeg')
recognize_faces_in_image('images_original/2.jpg')

it's hermione, the distance is 0.04725875
Not in the database, the distance is 0.8828622
Not in the database, the distance is 1.1699826
it's ron, the distance is 0.040145233
Not in the database, the distance is 1.0907894
Not in the database, the distance is 1.3107606
it's harry, the distance is 0.66620857
Not in the database, the distance is 1.3053504
